# Projeto Webnb

### Membros: Carolina Hirschheimer, Paulo Kim, Raphael Lahiry

In [1]:
# Bibliotecas úteis 
import numpy as np
from bs4 import BeautifulSoup
import requests
import pandas as pd
import seaborn as sns
import re
from tqdm import tqdm
from time import sleep


sns.set()

# Extração dos Dados

In [2]:


url = 'https://www.airbnb.com.br/s/S%C3%A3o-Paulo-~-SP/homes?tab_id=home_tab&query=S%C3%A3o%20Paulo%20-%20SP&place_id=ChIJ0WGkg4FEzpQRrlsz_whLqZs&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=june&flexible_trip_dates%5B%5D=may&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=calendar&disable_auto_translation=false&source=structured_search_input_header&search_type=autocomplete_click'
resposta = requests.get(url = url)
resposta.encoding = 'utf-8'

for i in tqdm(range(0,5), desc = 'Extraindo dados...'):    
    
    lista_titulo = []
    lista_bairro = []
    lista_imovel = []
    lista_preco = []
    lista_avaliacao = []
    lista_caracteristicas = []

    soup = BeautifulSoup(resposta.text, 'html.parser')
    
    anuncios = soup.findAll('div', class_='_8ssblpx') # anúncios


    for i in range(0, len(anuncios)):
        texto_caracteristicas = []

        anuncio = anuncios[i]

        titulo = anuncio.find('meta', itemprop='name').attrs['content']
        lista_titulo.append(titulo)

        bairro = anuncio.find('div', class_="_b14dlit").get_text().split(' em ')[1]
        lista_bairro.append(bairro)

        tipo_imovel = anuncio.find('div', class_="_b14dlit").get_text().split(' em ')[0]
        lista_imovel.append(tipo_imovel)

        caracteristicas = anuncio.findAll('span', class_="_3hmsj")[2:]
        for caracteristica in caracteristicas:
            texto_caracteristicas.append(caracteristica.get_text())
        lista_caracteristicas.append(texto_caracteristicas)

        preco = int(anuncio.find('span', class_="_155sga30").get_text()[2:])
        lista_preco.append(preco)

        avaliacao = anuncio.find('span', class_="_10fy1f8")
        if avaliacao is not None:
            avaliacao = float(anuncio.find('span', class_="_10fy1f8").get_text())
        else:
            avaliacao = np.nan
        lista_avaliacao.append(avaliacao)
        
    url = 'https://www.airbnb.com.br' + soup.find('a', class_="_za9j7e").attrs['href'] # próxima página
    resposta = requests.get(url = url)
    resposta.encoding = 'utf-8'

Extraindo dados...: 100%|████████████████████████████████████████████████████████████████| 5/5 [00:08<00:00,  1.65s/it]


In [3]:
lista_caracteristicas

[['2 hóspedes',
  '1 quarto',
  '1 cama',
  '1 banheiro',
  'Wi-Fi',
  'Estacionamento gratuito',
  'Cozinha',
  'Piscina'],
 ['4 hóspedes', '2 quartos', '2 camas', '1 banheiro', 'Wi-Fi', 'Cozinha'],
 ['2 hóspedes',
  '1 quarto',
  '1 cama',
  '1 banheiro',
  'Wi-Fi',
  'Cozinha',
  'Piscina'],
 ['2 hóspedes',
  '1 quarto',
  '1 cama',
  '1 banheiro',
  'Wi-Fi',
  'Cozinha',
  'Piscina'],
 ['2 hóspedes', '1 quarto', '1 cama', '1 banheiro', 'Wi-Fi', 'Piscina'],
 ['2 hóspedes',
  'Estúdio',
  '1 cama',
  '1 banheiro',
  'Wi-Fi',
  'Cozinha',
  'Piscina'],
 ['2 hóspedes',
  '1 quarto',
  '1 cama',
  '1 banheiro e meio',
  'Wi-Fi',
  'Estacionamento gratuito',
  'Cozinha',
  'Piscina'],
 ['2 hóspedes',
  '1 quarto',
  '1 cama',
  '1 banheiro',
  'Wi-Fi',
  'Cozinha',
  'Piscina'],
 ['2 hóspedes',
  '1 quarto',
  '1 cama',
  '1 banheiro',
  'Wi-Fi',
  'Cozinha',
  'Piscina'],
 ['2 hóspedes', '1 quarto', '1 cama', '1 banheiro', 'Wi-Fi', 'Piscina'],
 ['3 hóspedes',
  '1 quarto',
  '2 camas',


In [4]:
dados = pd.DataFrame({'Título': lista_titulo,
                      'Preço (R$)': lista_preco,
                      'Avaliação': lista_avaliacao,
                      'Bairro': lista_bairro,
                      'Imóvel': lista_imovel,
                      'Características': lista_caracteristicas
                     })

In [5]:
dados.head()

,Título,Preço (R$),Avaliação,Bairro,Imóvel,Características
0,Casa Estúdio Próx Av Paulista,137,4.82,Consolacao,Loft inteiro,"[2 hóspedes, 1 quarto, 1 cama, 1 banheiro, Wi-..."
1,Apartamento Completo 10 min do Brás e 25 de março,70,4.84,Campos Elíseos,Apartamento inteiro,"[4 hóspedes, 2 quartos, 2 camas, 1 banheiro, W..."
2,Studio Downtown São Luís -Centro de SP (C/Gara...,121,4.96,República,Apartamento inteiro,"[2 hóspedes, 1 quarto, 1 cama, 1 banheiro, Wi-..."
3,Studio deslumbrante em São Paulo próximo ao metrô,110,4.92,República,Flat inteiro,"[2 hóspedes, 1 quarto, 1 cama, 1 banheiro, Wi-..."
4,"Estúdio no Centro com Varanda, Wi-Fi & Smart-T...",77,4.82,Sé,Condomínio inteiro,"[2 hóspedes, 1 quarto, 1 cama, 1 banheiro, Wi-..."


In [6]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Título           20 non-null     object 
 1   Preço (R$)       20 non-null     int64  
 2   Avaliação        19 non-null     float64
 3   Bairro           20 non-null     object 
 4   Imóvel           20 non-null     object 
 5   Características  20 non-null     object 
dtypes: float64(1), int64(1), object(4)
memory usage: 1.1+ KB


In [7]:
dados['Características'][0]

['2 hóspedes',
 '1 quarto',
 '1 cama',
 '1 banheiro',
 'Wi-Fi',
 'Estacionamento gratuito',
 'Cozinha',
 'Piscina']